In [ ]:
import os
import pandas as pd
import numpy as np

from quick_pp.objects import Project

####  Steps to create "MOCK" project
1. Run the next cell.
2. Specify the example either clastic or carbonate.
3. MOCK.qppp project will be saved in notebooks\data\04_project folder.

* Note that the required curves in the LAS files are 'GR', 'RT', 'NPHI', 'RHOB'

In [ ]:
data_path = 'data/'
filenames = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        filenames.append(os.path.join(root, file)) if file.endswith('.las') and not 'seismic' in root  else None
print(filenames)

project_name = "CNS_ACORN"
project = Project(name=project_name)
project.read_las(filenames)
# clear_output()               

In [ ]:

tops_df = pd.read_excel(rf"data\OGA_UK_WellTopsDatabase.xlsx")

In [ ]:

data = project.get_all_data()

In [ ]:
data.WELL_NAME.unique()

In [ ]:
tops_df.head()

In [ ]:
# Process well tops data
data = project.get_all_data()
merged_df = data.copy()

tops_df = pd.read_excel(rf"data\OGA_UK_WellTopsDatabase.xlsx")
tops_df['WELL_NAME'] = tops_df['WELLREGNO'].str.replace('/', '-').str.replace(' ', '')
for well_name, well_data in data.groupby('WELL_NAME'):
    marker_df = tops_df[tops_df.WELL_NAME == well_name].copy()
    marker_df['DEPTH'] = marker_df['Depth_MD'].astype('float')
    marker_df['ZONES'] = marker_df['Top_Pick_Name']

    # Merge with well logs
    temp_df = pd.merge_asof(well_data, marker_df[['DEPTH', 'ZONES']],
                            direction='nearest', on='DEPTH', tolerance=0.1524 / 2)
    merged_df.loc[merged_df.WELL_NAME == well_name, ['ZONES']] = temp_df[['ZONES']]

In [ ]:
merged_df.ZONES.unique()

In [ ]:
marker_df.ZONES.unique()

In [ ]:
# Save the update data to the project
project.update_data(merged_df)
project.save()